COMPLEJIDAD DE DATOS.
Sklearn tiene una clase experimental llamada Iterative imputer:

que toma una clumna con datos faltantes y de manera iterativa prueba con otra columna

estimar los valores faltantes en un conjunto de datos utilizando información de otras características (o columnas). En este caso, IterativeImputer realiza la imputación de manera iterativa, modelando cada característica con valores faltantes como una función de las otras características.

utiliza regrsion lineal

In [1]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Crear un DataFrame de ejemplo con una columna de clase
data = {
    'clase': ['A', 'A', 'B', 'B', 'A', 'B'],
    'precio': [200000, np.nan, 250000, np.nan, 300000, np.nan],
    'tamaño': [150, 200, np.nan, 300, 250, np.nan],
    'número_de_habitaciones': [3, 4, 2, 5, 3, 4],
    'edad': [10, 5, 15, 20, 8, np.nan]
}

df = pd.DataFrame(data)

# Función para imputar valores en cada grupo
def imputar_por_clase(subdf):
    imputer = IterativeImputer(max_iter=10, random_state=0)
    return pd.DataFrame(imputer.fit_transform(subdf), columns=subdf.columns)

# Agrupar por clase y aplicar la imputación
df_imputed = df.groupby('clase').apply(lambda x: imputar_por_clase(x.drop(columns='clase')))

# Reindexar para mantener la estructura original
df_imputed = df_imputed.reset_index(level=0, drop=True)

# Agregar la columna de clase de nuevo
df_imputed['clase'] = df['clase']

print(df_imputed)


          precio  tamaño  número_de_habitaciones       edad clase
0  200000.000000   150.0                     3.0  10.000000     A
1  250079.968013   200.0                     4.0   5.000000     A
2  300000.000000   250.0                     3.0   8.000000     B
0  250000.000000   300.0                     2.0  15.000000     A
1  250000.000000   300.0                     5.0  20.000000     A
2  250000.000000   300.0                     4.0  18.333333     B


<ipython-input-1-dc7de29cbbc8>:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_imputed = df.groupby('clase').apply(lambda x: imputar_por_clase(x.drop(columns='clase')))


TAMBIEN EXISTE ALGO MAS ESTANDAR Y MAS SIMPLE LLAMDA SIMPLE IMPUTER
donde tu indicas las estrategia para rellenar datos.

In [1]:
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit([[1, 2], [np.nan, 3], [7, 6]])
SimpleImputer()
X = [[np.nan, 2], [6, np.nan], [7, 6]]
print(imp.transform(X))

[[4.         2.        ]
 [6.         3.66666667]
 [7.         6.        ]]


Cargar Librerias

In [7]:
import pandas as pd                  # A fundamental package for linear algebra and multidimensional arrays
import numpy as np                   # Data analysis and data manipulating tool
import random                        # Library to generate random numbers
from collections import Counter      # Collection is a Python module that implements specialized container datatypes providing
                                     # alternatives to Python’s general purpose built-in containers, dict, list, set, and tuple.
                                     # Counter is a dict subclass for counting hashable objects
# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# To ignore warnings in the notebook
import warnings
warnings.filterwarnings("ignore")

Cargar dataset

In [8]:
# This is a subset of the original data available at kaggle.
fraud_data = pd.read_csv("C:\\Users\\Polar\\Documents\\ESCUELA\\5TO_SEMESTRE\\MAKINITAS\\Practica4\\fraud_data.csv")

fraud_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2994681,0,242834,25.000,H,9803,583.0,150.0,visa,226.0,...,firefox 56.0,24.0,1920x1080,match_status:2,T,F,T,T,desktop,rv:56.0
1,3557242,0,15123000,117.000,W,7919,194.0,150.0,mastercard,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3327470,0,8378575,73.773,C,12778,500.0,185.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3118781,0,2607840,400.000,R,12316,548.0,150.0,visa,195.0,...,mobile safari generic,32.0,1136x640,match_status:2,T,F,T,F,mobile,iOS Device
4,3459772,0,12226544,31.950,W,9002,453.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Imprimir info del dataset.

In [9]:
fraud_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59054 entries, 0 to 59053
Columns: 434 entries, TransactionID to DeviceInfo
dtypes: float64(385), int64(18), object(31)
memory usage: 195.5+ MB


Hay 434 columnas y 59054 filas

In [10]:
# Taking a look at the target variable
fraud_data.isFraud.value_counts()       # The value_counts() function is used to get a Series containing counts of unique values.

isFraud
0    57049
1     2005
Name: count, dtype: int64

De todos esos datos, solo  2005 pertenecen a es fraude y 57049 no son fraudes, hay un desequilibrio gigante

Ahora vamos a revisar cuantos datos vacios tienen cada columna

In [11]:
fraud_data.isnull().sum() / len(fraud_data) * 100   # To get percentage of missing data in each column

TransactionID      0.000000
isFraud            0.000000
TransactionDT      0.000000
TransactionAmt     0.000000
ProductCD          0.000000
                    ...    
id_36             75.945745
id_37             75.945745
id_38             75.945745
DeviceType        75.979612
DeviceInfo        79.813391
Length: 434, dtype: float64

Para llenar las filas vacias no hay un patron que sirva ´para todas, en este caso primero vamos a eliminar las columnas que tengan mas del 20 porciento de informacion perdida

In [12]:
fraud_data = fraud_data[fraud_data.columns[fraud_data.isnull().mean() < 0.2]]    # Will keep those columns which has missing values less than 20%

Ahora para los datos numericos, llenaremos las columnas vacias con el maedia de la columna.

In [13]:
# filling missing values of numerical columns with mean value.
num_cols = fraud_data.select_dtypes(include=np.number).columns      # getting all the numerical columns

fraud_data[num_cols] = fraud_data[num_cols].fillna(fraud_data[num_cols].mean())   # fills the missing values with mean

Para las columnas que poseen cadenas, tomaremos la moda ya que no es un dato numerico.

In [14]:
cat_cols = fraud_data.select_dtypes(include = 'object').columns    # getting all the categorical columns

fraud_data[cat_cols] = fraud_data[cat_cols].fillna(fraud_data[cat_cols].mode().iloc[0])  # fills the missing values with maximum occuring element in the column

Ahora cuando se revisa si el dataset tiene huecos podemos ver que no los tienen.

In [15]:
# Let's have a look if there still exist any missing values
fraud_data.isnull().sum() / len(fraud_data) * 100

TransactionID     0.0
isFraud           0.0
TransactionDT     0.0
TransactionAmt    0.0
ProductCD         0.0
                 ... 
V317              0.0
V318              0.0
V319              0.0
V320              0.0
V321              0.0
Length: 182, dtype: float64

Ahora que la base de datos esta Compelta debemos solucionar el problema de el desequilibrio de clases, Lo primero que vamos a hacer es Aplicar el metodo One Hot Encoding con el proposito de volver las columnas con valores categoricas en valores numericos

In [16]:
fraud_data = pd.get_dummies(fraud_data, columns=cat_cols)    # earlier we have collected all the categorical columns in cat_cols
fraud_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,P_emaildomain_web.de,P_emaildomain_windstream.net,P_emaildomain_yahoo.co.jp,P_emaildomain_yahoo.co.uk,P_emaildomain_yahoo.com,P_emaildomain_yahoo.com.mx,P_emaildomain_yahoo.de,P_emaildomain_yahoo.es,P_emaildomain_yahoo.fr,P_emaildomain_ymail.com
0,2994681,0,242834,25.000,9803,583.0,150.0,226.0,269.0,87.0,...,False,False,False,False,True,False,False,False,False,False
1,3557242,0,15123000,117.000,7919,194.0,150.0,166.0,181.0,87.0,...,False,False,False,False,False,False,False,False,False,False
2,3327470,0,8378575,73.773,12778,500.0,185.0,224.0,284.0,60.0,...,False,False,False,False,False,False,False,False,False,False
3,3118781,0,2607840,400.000,12316,548.0,150.0,195.0,441.0,87.0,...,False,False,False,False,False,False,False,False,False,False
4,3459772,0,12226544,31.950,9002,453.0,150.0,226.0,264.0,87.0,...,False,False,False,False,True,False,False,False,False,False


X: Contiene todas las columnas del conjunto de datos, excepto la columna 'isFraud', lo que representa las características de entrada que describen los datos (variables independientes).
Y: Contiene solo la columna 'isFraud', que representa el objetivo a predecir (variable dependiente).

In [17]:

# Separate input features and output feature
X = fraud_data.drop(columns = ['isFraud'])       # input features
Y = fraud_data.isFraud      # output feature

Estandrizar datos para que esten en una misma metrica

In [18]:
from sklearn.preprocessing import StandardScaler
scaled_features = StandardScaler().fit_transform(X)
scaled_features = pd.DataFrame(data=scaled_features)
scaled_features.columns= X.columns
# Let's see how the data looks after scaling
scaled_features.head()

,TransactionID,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,C1,...,P_emaildomain_web.de,P_emaildomain_windstream.net,P_emaildomain_yahoo.co.jp,P_emaildomain_yahoo.co.uk,P_emaildomain_yahoo.com,P_emaildomain_yahoo.com.mx,P_emaildomain_yahoo.de,P_emaildomain_yahoo.es,P_emaildomain_yahoo.fr,P_emaildomain_ymail.com
0,-1.688548,-1.544958,-0.468203,-0.021940,1.412632,-0.286861,0.653753,-0.225982,0.077832,-0.099186,...,-0.021387,-0.022918,-0.009202,-0.004115,2.216281,-0.053413,-0.013649,-0.014839,-0.015399,-0.06253
1,1.615662,1.681426,-0.073540,-0.406928,-1.078794,-0.286861,-0.804662,-1.144356,0.077832,-0.099186,...,-0.021387,-0.022918,-0.009202,-0.004115,-0.451206,-0.053413,-0.013649,-0.014839,-0.015399,-0.06253
2,0.266093,0.219070,-0.258976,0.585989,0.881042,2.788641,0.605139,-0.069441,-10.788933,-0.099186,...,-0.021387,-0.022918,-0.009202,-0.004115,-0.451206,-0.053413,-0.013649,-0.014839,-0.015399,-0.06253
3,-0.959645,-1.032167,1.140478,0.491581,1.188468,-0.286861,-0.099761,1.569022,0.077832,-0.099186,...,-0.021387,-0.022918,-0.009202,-0.004115,-0.451206,-0.053413,-0.013649,-0.014839,-0.015399,-0.06253
4,1.043171,1.053404,-0.438389,-0.185621,0.580022,-0.286861,0.653753,-0.278162,0.077832,-0.082944,...,-0.021387,-0.022918,-0.009202,-0.004115,2.216281,-0.053413,-0.013649,-0.014839,-0.015399,-0.06253


Ahora se va a partir el dataset en un fragmento de entrenamiento y uno de testeo, con el porposito de evitar overfitting

en este caso se usara el 30 porciento del datasetpara pruebas

In [19]:
# Splitting the data
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

# X_train: independent feature data for training the model
# Y_train: dependent feature data for training the model
# X_test: independent feature data for testing the model; will be used to predict the target values
# Y_test: original target values of X_test; We will compare this values with our predicted values.

# test_size = 0.3: 30% of the data will go for test set and 70% of the data will go for train set
# random_state = 42: this will fix the split i.e. there will be same split for each time you run the code

**SMOTE**

metodo smote para manejar el desequilibrio de clases
que usa al algoritmo del vecino mas sercano para generar un nuevo dato sintetico

In [20]:
# import SMOTE
from imblearn.over_sampling import SMOTE

# create SMOTE object
sm = SMOTE(random_state=25, sampling_strategy=1.0)

# fit the sampling
X_train, Y_train = sm.fit_resample(X_train, Y_train)


In [21]:
np.unique(Y_train, return_counts=True)
# Y_train is numpy array, so unique() functions returns the count of all the unique elements in the array

(array([0, 1]), array([39942, 39942]))

**oversampling**


Ahora se hara uso de oversampling paraa llenar los datos faltantes de las clases  con sklearn

In [22]:
from sklearn.utils import resample

Reconstruir el dataset que se separo anteriormente

In [23]:
train_data = pd.concat([X_train, Y_train], axis = 1)

Separar clase minoritaria de la mayotitaria

In [24]:
# separate minority and majority class
not_fraud = train_data[train_data.isFraud==0]
fraud = train_data[train_data.isFraud==1]

Crear mas muestras de la clase con poco representantes para que se mepareje con la otra.

In [25]:
# Unsample minority; we are oversampling the minority class to match the number of majority classs
fraud_upsampled = resample(fraud,
                           replace = True, # Sample with replacement
                           n_samples = len(not_fraud), # Match number in majority class
                           random_state=27)

Combinar d enuevo

In [26]:
upsampled = pd.concat([not_fraud, fraud_upsampled])

Ver ahora cuantos datos existen de cada clase.

In [27]:
# Now let's check the classes count
upsampled.isFraud.value_counts()

isFraud
0    39942
1    39942
Name: count, dtype: int64